In [3]:
import openpyxl
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor

from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

from matplotlib import pyplot as plt
import datetime
import math
import pandas as pd
from scipy import stats
import seaborn as sn

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [5]:
# Get the airpointer data
airpointer_master = pd.read_pickle('airpointer_07Jun2020.pkl')

In [6]:
# First week of data from the egg sensors
egg_1_dataframe = pd.read_csv('2nd week of Egg Data/Egg#1/2020-05-16 Download_egg0080446e6e980130.csv')
egg_2_dataframe = pd.read_csv('2nd week of Egg Data/Egg#2/2020-05-16 Download_egg0080446fa82b0130.csv')
egg_4_dataframe = pd.read_csv('2nd week of Egg Data/Egg#4/2020-05-16 Download_egg00804426a6980132.csv')

In [14]:
# Read in the RAW DATA file
egg_1_dataframe = pd.read_csv('2nd week of Egg Data/Egg#1/2020-05-16 Download_egg0080446e6e980130.csv')
# Remove the Latitude, Longitude, and Altitude 
egg_1_dataframe = egg_1_dataframe.drop(columns=['Latitude[deg]', 'Longitude[deg]', 'Altitude[m]'])
# Convert the Timestamp column to an index and resample to 60 min averages
egg_1_dataframe['Timestamp'] = pd.to_datetime(egg_1_dataframe['Timestamp'])
egg_1_dataframe = egg_1_dataframe.resample('60min', on='Timestamp').mean()
# Write to a pickle file and CSV file
egg_1_dataframe.to_pickle('2020-05-16 Download_egg0080446e6e980130_sampled_hourly.pkl')
egg_1_dataframe.to_csv('2020-05-16 Download_egg0080446e6e980130_sampled_hourly.csv')

egg_1_dataframe

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa]
Timestamp,,,,,,,,,,
2020-05-06 13:00:00,20.520000,20.116667,3.166667,3.666667,4.000000,142.300000,1.421641,1935.236667,1.410120,99261.900000
2020-05-06 14:00:00,17.968333,24.757292,13.868750,15.925000,16.810417,63.958958,1.423080,263.038333,1.422146,99264.718750
2020-05-06 15:00:00,22.348833,19.432667,1.673333,2.333333,2.475000,60.012667,1.423583,95.994500,1.423568,99188.643333
2020-05-06 16:00:00,22.438667,18.594000,1.186667,1.920000,2.090000,61.118000,1.423567,56.341500,1.423861,99138.068333
2020-05-06 17:00:00,22.471500,18.093833,1.471667,2.248333,2.441667,62.872167,1.423533,49.240500,1.423915,99108.190000
...,...,...,...,...,...,...,...,...,...,...
2020-05-16 06:00:00,12.611000,65.638167,7.031667,9.443333,9.798333,46.910167,1.422959,60.441500,1.423359,100010.598333
2020-05-16 07:00:00,13.125167,64.307833,4.645000,6.471667,6.693333,45.372167,1.423030,65.634167,1.423342,100115.695000
2020-05-16 08:00:00,18.805500,49.320167,4.266667,5.765000,5.975000,39.189167,1.423687,65.547500,1.423618,100153.883333


In [19]:
# Read in the RAW DATA file
egg_2_dataframe = pd.read_csv('2nd week of Egg Data/Egg#2/2020-05-16 Download_egg0080446fa82b0130.csv')
# Remove the Latitude, Longitude, and Altitude 
egg_2_dataframe = egg_2_dataframe.drop(columns=['Latitude[deg]', 'Longitude[deg]', 'Altitude[m]'])
# Convert the Timestamp column to an index and resample to 60 min averages
egg_2_dataframe['Timestamp'] = pd.to_datetime(egg_2_dataframe['Timestamp'])
egg_2_dataframe = egg_2_dataframe.resample('60min', on='Timestamp').mean()
# Write to a pickle file and CSV file
egg_2_dataframe.to_pickle('2020-05-16 Download_egg0080446fa82b0130_sampled_hourly.pkl')
egg_2_dataframe.to_csv('2020-05-16 Download_egg0080446fa82b0130_sampled_hourly.csv')

egg_2_dataframe

,Temperature[degC],Humidity[percent],PM1.0[ug/m^3],PM2.5[ug/m^3],PM10.0[ug/m^3],O3[ppb],O3[V],NO2[ppb],NO2[V],Pressure[Pa]
Timestamp,,,,,,,,,,
2020-05-06 13:00:00,21.180000,23.270000,1.000000,1.500000,1.500000,132.520000,1.424859,2664.450000,1.397469,99291.800000
2020-05-06 14:00:00,18.300385,24.459423,1.203846,2.282692,2.573077,57.853654,1.426289,380.277692,1.414856,99296.513462
2020-05-06 15:00:00,21.573667,19.899667,1.035000,1.966667,2.193333,49.278667,1.426699,166.483333,1.416678,99223.611667
2020-05-06 16:00:00,20.936500,19.845833,1.188333,2.235000,2.576667,50.088667,1.426638,159.735500,1.416693,99177.978333
2020-05-06 17:00:00,20.838500,19.497333,1.536667,2.705000,3.088333,49.248667,1.426650,146.279333,1.416790,99149.185000
...,...,...,...,...,...,...,...,...,...,...
2020-05-16 06:00:00,12.435000,66.875500,5.360000,8.130000,8.476667,54.497833,1.425976,118.839000,1.416522,100042.838333
2020-05-16 07:00:00,12.901333,65.607333,4.170000,6.421667,6.716667,53.572667,1.426021,122.426833,1.416514,100148.910000
2020-05-16 08:00:00,19.509333,47.660500,3.545000,5.198333,5.418333,49.597333,1.426552,130.123167,1.416837,100183.331667


In [1]:
# Read in the RAW DATA file
egg_4_dataframe = pd.read_csv('2nd week of Egg Data/Egg#4/2020-05-16 Download_egg00804426a6980132.csv')
# Remove the Latitude, Longitude, and Altitude 
egg_4_dataframe = egg_4_dataframe.drop(columns=['Latitude[deg]', 'Longitude[deg]', 'Altitude[m]'])
# Convert the Timestamp column to an index and resample to 60 min averages
egg_4_dataframe['Timestamp'] = pd.to_datetime(egg_4_dataframe['Timestamp'])
egg_4_dataframe = egg_4_dataframe.resample('60min', on='Timestamp').mean()
# Write to a pickle file and CSV file
egg_4_dataframe.to_pickle('2020-05-16 Download_egg00804426a6980132_sampled_hourly.pkl')
egg_4_dataframe.to_csv('2020-05-16 Download_egg00804426a6980132_sampled_hourly.csv')

egg_4_dataframe

NameError: name 'pd' is not defined